In [26]:
# Importamos librerías.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
# sklearn.
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.preprocessing import label_binarize
# Hypertuning utilizando grid search.
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# Oversampler.
from imblearn.over_sampling import  RandomOverSampler
datos_url_pc= "C://Users/maxib/Source/repos/proyecto_final_coder/datos/Base_predictive_maintenance.csv"

In [27]:
# Leer .csv y creo un DF.
df = pd.read_csv(datos_url_pc, delimiter=",", index_col= ["UDI", "Product_ID"])
df.head()

,,Type,Air_temperature_[K],Process_temperature_[K],Rotational_speed_[rpm],Torque_[Nm],Tool_wear_[min],Target,Failure_Type
UDI,Product_ID,,,,,,,,
1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


# Definimos Variables 

In [28]:
# Definimos datos base.
data_x= df.drop(["Failure_Type", "Target"], axis=1)
data_y_ft= df["Failure_Type"]
data_y_t= df["Target"]
# Datos para testear.
data_test_x= pd.get_dummies(data_x)
data_test_y_ft_dum= pd.get_dummies(data_y_ft)

## Resampleo variable Failure type

In [29]:
# Oversampleamos variable Failure Type.
# Preparamos variable Target para entrenar los algoritmos.
X= data_x
y= data_y_ft
# Instantiating the random over sampler.
ros = RandomOverSampler(sampling_strategy={"Heat Dissipation Failure":1000,"Random Failures":1000,"Tool Wear Failure":1000,
                                            "Overstrain Failure":1000, "Power Failure":1000})
# Resampling X, y
X_ros_failure, y_ros_failure = ros.fit_resample(X, y)
X_ros_failure.info()
X_ros_failure["Type"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14652 entries, 0 to 14651
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Type                     14652 non-null  object 
 1   Air_temperature_[K]      14652 non-null  float64
 2   Process_temperature_[K]  14652 non-null  float64
 3   Rotational_speed_[rpm]   14652 non-null  int64  
 4   Torque_[Nm]              14652 non-null  float64
 5   Tool_wear_[min]          14652 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 686.9+ KB


array(['M', 'L', 'H'], dtype=object)

In [30]:
# Adquerimos Dummies de Failure Type (Para Train)
data_y_dum_train= pd.get_dummies(y_ros_failure)
data_X= pd.get_dummies(X_ros_failure)
data_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14652 entries, 0 to 14651
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Air_temperature_[K]      14652 non-null  float64
 1   Process_temperature_[K]  14652 non-null  float64
 2   Rotational_speed_[rpm]   14652 non-null  int64  
 3   Torque_[Nm]              14652 non-null  float64
 4   Tool_wear_[min]          14652 non-null  int64  
 5   Type_H                   14652 non-null  uint8  
 6   Type_L                   14652 non-null  uint8  
 7   Type_M                   14652 non-null  uint8  
dtypes: float64(3), int64(2), uint8(3)
memory usage: 615.4 KB


# Randon Forest Hiper Failure Type

In [39]:
rf= RandomForestClassifier() # Instanciamos el modelo
# UNO VS EL RESTO
model_rf= OneVsRestClassifier(rf)

# Definimos los parámetros de búsqueda
params ={ 
    'n_estimators': [200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [6,7,8],
    'criterion' :['entropy']
}
# metricas= {'f1_score': make_scorer(f1_score),
#            'precision_score': make_scorer(metrics.precision_score) ,
#            'recall_score': make_scorer(recall_score) }
grid_random_forest = GridSearchCV(estimator = rf,
                                  param_grid = params,
                                  scoring = "f1",
                                #   refit=f1_score,
                                  cv = 5, 
                                  verbose = 1, # Muestra el resultado en pantalla
                                  n_jobs = -1) # corrida en paralelo

In [33]:
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [40]:
grid_random_forest.fit(data_X, data_y_dum_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\maxib\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [6, 7, 8],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [200, 500]},
             scoring='f1', verbose=1)

In [36]:
#Obtenemos el mejor modelo!
grid_random_forest.best_estimator_
print("Mejores parametros: "+str(grid_random_forest.best_params_))
print("Mejor Score: "+str(grid_random_forest.best_score_)+'\n')
scores = pd.DataFrame(grid_random_forest.cv_results_)
scores

Mejores parametros: {'criterion': 'gini', 'max_depth': 4, 'max_features': 'auto', 'n_estimators': 200}
Mejor Score: nan



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.018290,0.806665,0.445264,0.111991,gini,4,auto,200,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,11.077254,1.490724,1.184875,0.324601,gini,4,auto,500,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,32
2,4.820767,1.120570,0.569617,0.237671,gini,4,sqrt,200,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
3,10.799013,1.679494,1.091371,0.207948,gini,4,sqrt,500,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,34
4,4.588750,0.719880,0.394358,0.058377,gini,4,log2,200,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,35
5,11.531189,1.151163,0.978739,0.105266,gini,4,log2,500,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,36
6,4.410791,0.664193,0.487474,0.120298,gini,5,auto,200,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
7,10.225503,0.915887,1.053659,0.151907,gini,5,auto,500,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,38
8,4.183174,0.482989,0.400691,0.042245,gini,5,sqrt,200,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,39
9,10.755368,0.989884,0.945005,0.144804,gini,5,sqrt,500,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,40


In [41]:
#Predicción de casos nuevos
# Prediccion en Train
y_train_pred= grid_random_forest.predict(data_X)
# Prediccion en Test
y_test_pred= grid_random_forest.predict(data_test_x)
# 
y_score= grid_random_forest.best_estimator_.predict_proba(data_test_x)
#Accuracy
print('Exactitud:', accuracy_score(data_test_y_ft_dum, y_test_pred))

Exactitud: 0.9416


In [42]:
# Analisis score F1
print(f"F1 Score TEST del classificador macro: {f1_score(data_test_y_ft_dum, y_test_pred, average='macro')}")

print(f"F1 Score TEST del classificador micro: {f1_score(data_test_y_ft_dum, y_test_pred, average='micro')}")

print(f"F1 Score TEST del classificador weighted: {f1_score(data_test_y_ft_dum, y_test_pred, average='weighted')}")

print(f"F1 Score TEST del classificador: {f1_score(data_test_y_ft_dum, y_test_pred, average=None)}")
# Analisis jaccard
print(f"Jaccard Score TEST del classificador macro: {metrics.jaccard_score(data_test_y_ft_dum, y_test_pred, average='macro')}")

print(f"Jaccard Score TEST del classificador micro: {metrics.jaccard_score(data_test_y_ft_dum, y_test_pred, average='micro')}")

print(f"Jaccard Score TEST del classificador weighted: {metrics.jaccard_score(data_test_y_ft_dum, y_test_pred, average='weighted')}")


F1 Score TEST del classificador macro: 0.712428627833864
F1 Score TEST del classificador micro: 0.9612086565945285
F1 Score TEST del classificador weighted: 0.962549392275697
F1 Score TEST del classificador: [0.70860927 0.97073534 0.78723404 0.87292818 0.36363636 0.57142857]
Jaccard Score TEST del classificador macro: 0.5896179340077273
Jaccard Score TEST del classificador micro: 0.925314465408805
Jaccard Score TEST del classificador weighted: 0.9310803724132986


In [43]:
metrics.classification_report(data_test_y_ft_dum, y_test_pred)

C:\Users\maxib\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.56      0.96      0.71       112\n           1       1.00      0.95      0.97      9652\n           2       0.67      0.95      0.79        78\n           3       0.92      0.83      0.87        95\n           4       1.00      0.22      0.36        18\n           5       0.50      0.67      0.57        45\n\n   micro avg       0.98      0.94      0.96     10000\n   macro avg       0.78      0.76      0.71     10000\nweighted avg       0.99      0.94      0.96     10000\n samples avg       0.94      0.94      0.94     10000\n'